In [1]:
from __future__ import print_function
import eeglearn
from eeglearn import eeg_cnn_lib1
from eeglearn.eeg_cnn_lib1 import *


import numpy as np
import scipy.io
from functools import reduce

C:\ProgramData\Anaconda3\envs\my_env27\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
# Load electrode locations
print('Loading data...')
locs = scipy.io.loadmat('C:/Users/alena/Desktop/homed/laba/normData/locations.mat')
locs_3d = locs['locs']
locs_2d = []
elNum = locs_3d.shape[0]  # Number of electrodes
# Convert to 2D
for e in locs_3d:
    locs_2d.append(azim_proj(e))

Loading data...


In [3]:
feats = scipy.io.loadmat('C:/Users/alena/Desktop/homed/laba/dataSep/FeatureMat_timeWin35_36_37.mat')['features']
subj_nums = np.squeeze(scipy.io.loadmat('C:/Users/alena/Desktop/homed/laba/dataSep/trials_subNums35_36_37.mat')['subjectNum'])
# Leave-Subject-Out cross validation
fold_pairs = []  # Pairs of test and train data for validation
for i in np.unique(subj_nums):
    ts = subj_nums == i
    tr = np.squeeze(np.nonzero(np.bitwise_not(ts)))  # Indeces of trials corresponding to all  subjects except for i-th
    ts = np.squeeze(np.nonzero(ts))  # Not(tr)
    np.random.shuffle(tr)  # Shuffle indices
    np.random.shuffle(ts)
    fold_pairs.append((tr, ts))

# CNN Mode
print('Generating images...')
# Find the average response over time windows
av_feats = reduce(lambda x, y: x + y,
                  [feats[:, i * 3 * elNum:(i + 1) * 3 * elNum] for i in range(feats.shape[1] / (3 * elNum))])
av_feats = av_feats / (feats.shape[1] / (3 * elNum))
images = gen_images(np.array(locs_2d),
                    av_feats,
                    32, normalize=False)
print('\n')

Generating images...


Interpolating 1849/1849Interpolating 1503/1849Interpolating 1507/1849Interpolating 1511/1849Interpolating 1515/1849Interpolating 1519/1849Interpolating 1523/1849Interpolating 1527/1849Interpolating 1531/1849Interpolating 1535/1849Interpolating 1539/1849Interpolating 1543/1849Interpolating 1547/1849Interpolating 1551/1849Interpolating 1555/1849Interpolating 1559/1849Interpolating 1563/1849Interpolating 1567/1849Interpolating 1571/1849Interpolating 1575/1849Interpolating 1579/1849Interpolating 1583/1849Interpolating 1587/1849Interpolating 1591/1849Interpolating 1595/1849Interpolating 1599/1849Interpolating 1603/1849Interpolating 1607/1849Interpolating 1611/1849Interpolating 1615/1849Interpolating 1619/1849Interpolating 1623/1849Interpolating 1627/1849Interpolating 1631/1849Interpolating 1635/1849Interpolating 1637/1849Interpolating 1640/1849Interpolating 1642/1849Interpolating 1644/1849Interpolating 1649/1849Interpolating 1653/1849Interpolating 1657/1849Interpolating 1661/1849Interpolati

In [7]:
print('Training the CNN Model...')
# Class labels should start from 0 (so here 1 is subtracted)
train_without_test(images, np.squeeze(feats[:, -1]) - 1, fold_pairs[2], 'cnn',
                   'C:/Users/alena/Desktop/homed/laba/results/e25lr0005part/',
                   learning_rate=0.0005, num_epochs=25)

Training the CNN Model...
Building model and compiling functions...
Starting training...
Epoch 1 of 25 took 46.670s
  training loss:		0.680124
  validation loss:		0.696670
  validation accuracy:		53.49 %
  validation auc:		0.67373
Epoch 2 of 25 took 47.196s
  training loss:		0.693159
  validation loss:		0.649729
  validation accuracy:		64.54 %
  validation auc:		0.65973
Epoch 3 of 25 took 50.640s
  training loss:		0.608021
  validation loss:		0.671409
  validation accuracy:		63.24 %
  validation auc:		0.67325
Epoch 4 of 25 took 52.206s
  training loss:		0.629926
  validation loss:		0.641310
  validation accuracy:		64.69 %
  validation auc:		0.67228
Epoch 5 of 25 took 48.710s
  training loss:		0.623786
  validation loss:		0.638762
  validation accuracy:		65.82 %
  validation auc:		0.68543
Epoch 6 of 25 took 48.728s
  training loss:		0.697451
  validation loss:		0.643779
  validation accuracy:		66.41 %
  validation auc:		0.68467
Epoch 7 of 25 took 49.389s
  training loss:		0.666454
  val

IOError: [Errno 2] No such file or directory: 'C:/Users/alena/Desktop/homed/laba/results/e25lr0005part/ROC_val_cnn25.png'

In [10]:
epochs = np.arange(1,26)
val_auc = np.array([0.67373, 0.65973, 0.67325, 0.67228, 0.68543, 0.68467, 0.68700, 0.68498, 0.68136, 0.68902,
                   0.68787, 0.68662, 0.69515, 0.69311, 0.68097, 0.69158, 0.70235, 0.69069, 0.70303, 0.70757, 
                   0.71065, 0.71009, 0.70608, 0.70911, 0.71663])
train_loss = np.array([0.680124, 0.693159, 0.608021, 0.629926, 0.623786, 0.697451, 0.666454, 0.650261, 0.638609, 0.633161,
                    0.601026, 0.612877, 0.642622, 0.636036, 0.622586, 0.609573, 0.593381, 0.612779, 0.599316, 0.588602,
                    0.611423, 0.591099, 0.576597,0.635123, 0.589366])
val_loss = np.array([0.696670, 0.649729, 0.671409, 0.641310, 0.638762, 0.643779, 0.645546, 0.651217, 0.636884, 0.633776,
                     0.634359, 0.653946, 0.630919, 0.638730, 0.638858, 0.629481, 0.649667, 0.631230, 0.635616, 0.636359,
                     0.628865, 0.626870, 0.644334, 0.624906, 0.623840])
val_accu = np.array([53.49, 64.54, 63.24, 64.69, 65.82, 66.41, 65.05, 65.35, 65.41, 63.78, 64.20, 63.88, 64.67, 67.18,
                     63.97, 64.50, 66.58, 65.07, 65.07, 64.50, 66.56, 65.99, 67.03, 65.24, 66.24])

# Plot AUC
plt.plot(epochs, val_auc)
plt.xlabel('epochs')
plt.title('AUC in ' + model_type + ' model (validation)')
plt.savefig(dir_name + 'AUC_val_' + model_type + str(num_epochs) + '.png')

plt.cla()
plt.clf()

# Plot accuracy
plt.plot(epochs, val_accu)
plt.xlabel('epochs')
plt.ylabel('%')
plt.title('Accuracy in ' + model_type + ' model (validation)')
plt.savefig(dir_name + 'Acc_val_' + model_type + str(num_epochs) + '.png')

plt.cla()
plt.clf()

# Plot loss
t_loss, = plt.plot(epochs, train_loss, label='train loss')
v_loss, = plt.plot(epochs, val_loss, label='validation loss')
plt.legend(handles=[t_loss, v_loss])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Loss in ' + model_type + ' model')
plt.savefig(dir_name + 'loss_' + model_type + str(num_epochs) + '.png')

plt.cla()
plt.clf()
plt.close()

NameError: name 'y_val' is not defined

In [4]:
# Conv-LSTM Mode
print('Generating images for all time windows...')
images_timewin = np.array([gen_images(np.array(locs_2d),
                                      feats[:, i * (3 * elNum):(i + 1) * (3 * elNum)], 32, normalize=False) for i in
                           range(feats.shape[1] / (3 * elNum))
                           ])
print('\n')

Generating images for all time windows...


Interpolating 1098/1849Interpolating 1472/1849Interpolating 1476/1849Interpolating 1480/1849Interpolating 1484/1849Interpolating 1488/1849Interpolating 1492/1849Interpolating 1496/1849Interpolating 1500/1849Interpolating 1504/1849Interpolating 1508/1849Interpolating 1512/1849Interpolating 1516/1849Interpolating 1520/1849Interpolating 1524/1849Interpolating 1528/1849Interpolating 1532/1849Interpolating 1536/1849Interpolating 1540/1849Interpolating 1544/1849Interpolating 1548/1849Interpolating 1552/1849Interpolating 1556/1849Interpolating 1560/1849Interpolating 1566/1849Interpolating 1570/1849Interpolating 1574/1849Interpolating 1578/1849Interpolating 1582/1849Interpolating 1586/1849Interpolating 1590/1849Interpolating 1594/1849Interpolating 1598/1849Interpolating 1602/1849Interpolating 1606/1849Interpolating 1609/1849Interpolating 1613/1849Interpolating 1617/1849Interpolating 1621/1849Interpolating 1625/1849Interpolating 1629/1849Interpolating 1633/1849Interpolating 1637/1849Interpolati

Interpolating 647/18499Interpolating 1107/1849Interpolating 1110/1849Interpolating 1114/1849Interpolating 1118/1849Interpolating 1122/1849Interpolating 1126/1849Interpolating 1130/1849Interpolating 1134/1849Interpolating 1138/1849Interpolating 1142/1849Interpolating 1146/1849Interpolating 1150/1849Interpolating 1154/1849Interpolating 1158/1849Interpolating 1162/1849Interpolating 1165/1849Interpolating 1168/1849Interpolating 1172/1849Interpolating 1176/1849Interpolating 1180/1849Interpolating 1184/1849Interpolating 1188/1849Interpolating 1192/1849Interpolating 1196/1849Interpolating 1200/1849Interpolating 1203/1849Interpolating 1207/1849Interpolating 1211/1849Interpolating 1215/1849Interpolating 1219/1849Interpolating 1223/1849Interpolating 1227/1849Interpolating 1231/1849Interpolating 1235/1849Interpolating 1239/1849Interpolating 1243/1849Interpolating 1246/1849Interpolating 1250/1849Interpolating 1254/1849Interpolating 1258/1849Interpolating 1262/1849Interpolating 1266/1849Interpolati

Interpolating 1848/1849nterpolating 656/1849Interpolating 660/1849Interpolating 664/1849Interpolating 668/1849Interpolating 672/1849Interpolating 676/1849Interpolating 680/1849Interpolating 684/1849Interpolating 688/1849Interpolating 692/1849Interpolating 696/1849Interpolating 700/1849Interpolating 704/1849Interpolating 708/1849Interpolating 712/1849Interpolating 716/1849Interpolating 720/1849Interpolating 724/1849Interpolating 728/1849Interpolating 732/1849Interpolating 736/1849Interpolating 740/1849Interpolating 744/1849Interpolating 748/1849Interpolating 752/1849Interpolating 756/1849Interpolating 760/1849Interpolating 764/1849Interpolating 768/1849Interpolating 771/1849Interpolating 774/1849Interpolating 777/1849Interpolating 780/1849Interpolating 787/1849Interpolating 791/1849Interpolating 795/1849Interpolating 799/1849Interpolating 803/1849Interpolating 807/1849Interpolating 811/1849Interpolating 815/1849Interpolating 819/1849Interpolating 823/1849Interpolating 827/1849Interpolat

In [9]:
print('Training the LSTM-CONV Model...')
train_without_test(images_timewin, np.squeeze(feats[:, -1]) - 1, fold_pairs[2], 'mix',
                   'C:/Users/alena/Desktop/homed/laba/results/e25lr0005part/',
                   learning_rate=0.0005, num_epochs=25)

print('Done!')

Training the LSTM-CONV Model...
Building model and compiling functions...
Starting training...
Epoch 1 of 25 took 155.333s
  training loss:		0.645360
  validation loss:		0.667216
  validation accuracy:		64.69 %
  validation auc:		0.67424
Epoch 2 of 25 took 161.633s
  training loss:		0.643319
  validation loss:		0.647222
  validation accuracy:		65.11 %
  validation auc:		0.68514
Epoch 3 of 25 took 159.197s
  training loss:		0.617426
  validation loss:		0.649482
  validation accuracy:		63.31 %
  validation auc:		0.68430
Epoch 4 of 25 took 161.493s
  training loss:		0.618537
  validation loss:		0.735940
  validation accuracy:		59.61 %
  validation auc:		0.68310
Epoch 5 of 25 took 162.566s
  training loss:		0.594627
  validation loss:		0.675274
  validation accuracy:		64.97 %
  validation auc:		0.68750
Epoch 6 of 25 took 165.865s
  training loss:		0.589040
  validation loss:		0.682112
  validation accuracy:		64.07 %
  validation auc:		0.68950
Epoch 7 of 25 took 173.783s
  training loss:		0